# Метод опорных векторов. Ядра.


In [2]:
import numpy as np
import pandas as pd
import scipy as sp
import sklearn
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR
#plots
import matplotlib.pyplot as plt
%matplotlib inline
#beautiful plots
import seaborn as sns
#linear regression
import statsmodels.api as sm
#set style for plots
sns.set_style('darkgrid')
#off the warnings
import warnings
warnings.filterwarnings("ignore")

## 1. Метод Опорных Векторов и Задача Регрессии

### Набор данных с ценами на недвижимость (*train.csv*)
* Ранее мы искали линейную зависимость между зависимой переменной (Y, Sale Price) и некоторым набором признаков (X)

Подробнее о наборе данных https://www.kaggle.com/datasets/lespin/house-prices-dataset

In [3]:
cols=['Id', 'MSSubClass', 'LotArea', 'OverallQual',\
      'OverallCond', 'YearBuilt', 'YearRemodAdd',\
      'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',\
     'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',\
     'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',\
     'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',\
     'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',\
     'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']

In [ ]:
data = pd.read_csv('train.csv', index_col=0, usecols=cols)
data.head()

#### Корреляционная матрица:

In [ ]:
dt=data.copy()
sns.set(style="white")

# Вычислить корреляционную матрицу
corr = dt.corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(11, 9))

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

### Наилучшая модель, которая у нас получилась, выглядела так:

In [6]:
X = data.drop(['SalePrice', 'GrLivArea'], axis=1)
Y = data.SalePrice

X = sm.add_constant(X)

model = sm.OLS(Y, X)

results = model.fit()

In [7]:
cols2 = ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtUnfSF',\
         'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath',\
 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',\
 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']

In [8]:
alpha = 0.05

# исключение признаков
while results.pvalues[results.pvalues > 0.05].shape[0] > 0:
    use_cols=results.pvalues[results.pvalues != results.pvalues.max()].keys().tolist()
    X = X[use_cols]
    Y = data.SalePrice

    model = sm.OLS(Y, X)

    results = model.fit()

In [ ]:
print(results.summary2())

### Давайте попробуем смоделировать зависимость при помощи линейного SVM

* Какие параметры есть у sklearn-реализации SVR?

In [3]:
SVR?

* Зададим параметры для SVR "из коробки":

In [11]:
regressor = SVR(kernel='linear', verbose=True)

In [ ]:
X = data.drop('SalePrice', axis=1)
model = regressor.fit(X,Y)

In [13]:
predictions = model.predict(X)

In [ ]:
sklearn.metrics.r2_score(Y, predictions)

In [ ]:
model.score(X,Y)

* Можно ли сделать "лучше", не переходя к ядрам? 

Попробуем варьировать дефолтные параметры:

In [20]:
C = [0.1, 1, 10, 50, 100]
epsilon = [0.1, 1, 10]

In [ ]:
regressor = SVR(kernel='linear', C=5, epsilon=0.01)
model = regressor.fit(X,Y)
predictions = model.predict(X)
sklearn.metrics.r2_score(Y, predictions)

* А теперь давайте посмотрим, как изменится ситуация при использовании ядер:

### SVM c ядрами

* для некоторых ядер требуется "стандартизировать" данные
* два основных способа: нормализация данных и преобразование к интервалу [0, 1]

In [46]:
# нормализация
def scale(df):
    return ((df - df.mean()) / df.std())

# преобразование к интервалу [0, 1]
def scaling_to_zero_one(df):
    return (df - df.min()) / (df.max() - df.min())

* что будет без нормализации? **rbf-ядро**

In [ ]:
regressor = SVR(kernel='rbf')
model = regressor.fit(X,Y)
predictions = model.predict(X)
sklearn.metrics.r2_score(Y, predictions)

* а с нормализацией:

In [ ]:
regressor = SVR(kernel='rbf')
model = regressor.fit(scaling_to_zero_one(X),scaling_to_zero_one(Y))
predictions = model.predict(scaling_to_zero_one(X))
sklearn.metrics.r2_score(scaling_to_zero_one(Y), predictions)

In [ ]:
regressor = SVR(kernel='rbf')
model = regressor.fit(scale(X),scale(Y))
predictions = model.predict(scale(X))
sklearn.metrics.r2_score(scale(Y), predictions)

* другие ядра

In [54]:
SVR?

In [ ]:
regressor = SVR(kernel='poly')
model = regressor.fit(scale(X),scale(Y))
predictions = model.predict(scale(X))
sklearn.metrics.r2_score(scale(Y), predictions)

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(scale(Y), predictions)
#plt.plot(scaling_to_zero_one(np.arange(Y.shape[0])), scaling_to_zero_one(np.arange(Y.shape[0])))
plt.title('True values vs Predictions')
plt.xlabel('True values')
plt.ylabel('Predictions')
plt.show()

In [ ]:
regressor = SVR(kernel='poly')
model = regressor.fit(scaling_to_zero_one(X),scaling_to_zero_one(Y))
predictions = model.predict(scaling_to_zero_one(X))
sklearn.metrics.r2_score(scaling_to_zero_one(Y), predictions)

In [ ]:
regressor = SVR(kernel='sigmoid')
model = regressor.fit(scale(X),scale(Y))
predictions = model.predict(scale(X))
sklearn.metrics.r2_score(scale(Y), predictions)

In [ ]:
regressor = SVR(kernel='sigmoid')
model = regressor.fit(scaling_to_zero_one(X),scaling_to_zero_one(Y))
predictions = model.predict(scaling_to_zero_one(X))
sklearn.metrics.r2_score(scaling_to_zero_one(Y), predictions)

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(scaling_to_zero_one(Y), predictions)
plt.plot(scaling_to_zero_one(np.arange(Y.shape[0])), scaling_to_zero_one(np.arange(Y.shape[0])))
plt.title('True values vs Predictions')
plt.xlabel('True values')
plt.ylabel('Predictions')
plt.show()

### **Задание 1**

Подобрать конфигурацию SVR (с ядром или без ядра), которая наилучшим образом моделирует существующую зависимость.
* подобрать ядро
* подобрать параметры

## 2. Метод опорных векторов для задачи классификации

В прошлый раз мы работали с набором данных "ирисы":

In [ ]:
X, Y = sklearn.datasets.load_iris(return_X_y=True)
names = ['sepal length', 'sepal width', 'petal length', 'petal width']
classes = ['setosa', 'versicolor', 'virginica']

#создать объект pandas

X = pd.DataFrame(X, columns=names)
X.head()

Целевая переменная уже закодирована:

    * 0 - Setosa

    * 1 - Versicolor
    
    * 2 - Verginica


Корреляция:

In [ ]:
dt=pd.concat([X,pd.DataFrame(Y, columns=['target'])], axis=1)
sns.set(style="white")

corr = dt.corr()

mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

f, ax = plt.subplots(figsize=(11, 9))

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.show()

### Графики

In [ ]:
sns.pairplot(dt, kind='scatter', hue='target')
plt.show()

**Обычная логистическая регрессия**

In [ ]:
#from sklearn import metrics 
logit = sm.MNLogit(Y, X)
result = logit.fit()
preds=np.argmax(result.predict(X).values,axis=1)
accuracy_score(Y,preds)

**Логистическая регрессия с L1 регуляризацией**

In [ ]:
from sklearn import linear_model
model=sklearn.linear_model.LogisticRegression(penalty='l1',multi_class='multinomial', solver='saga').fit(X,Y)
preds=model.predict(X)
accuracy_score(Y, preds)

**Логистическая регрессия с L2 регуляризацией**

In [ ]:
from sklearn import linear_model
model=sklearn.linear_model.LogisticRegression(penalty='l2',multi_class='multinomial', solver='saga', fit_intercept=True).fit(X,Y)
preds=model.predict(X)
accuracy_score(Y, preds)

### SVC

In [92]:
clf = sklearn.svm.SVC(kernel='linear')

In [93]:
model = clf.fit(X,Y)

In [96]:
predictions = model.predict(X)

In [ ]:
model.score(X,Y)

In [ ]:
accuracy_score(Y, predictions)

* Что с ядрами?

In [ ]:
clf = sklearn.svm.SVC(kernel='rbf')
model = clf.fit(X, Y)
predictions = model.predict(X)
accuracy_score(Y, predictions)

* Почему "работает" без стандартизации?

In [ ]:
X.head()

In [ ]:
X.describe()

* А если добавить стандартизацию?

In [ ]:
clf = sklearn.svm.SVC(kernel='rbf')
model = clf.fit(scale(X), Y)
predictions = model.predict(scale(X))
accuracy_score(Y, predictions)

In [ ]:
clf = sklearn.svm.SVC(kernel='rbf')
model = clf.fit(scaling_to_zero_one(X), Y)
predictions = model.predict(scaling_to_zero_one(X))
accuracy_score(Y, predictions)

* А теперь другие ядра:

In [ ]:
clf = sklearn.svm.SVC(kernel='poly')
model = clf.fit(X, Y)
predictions = model.predict(X)
accuracy_score(Y, predictions)

In [ ]:
clf = sklearn.svm.SVC(kernel='poly')
model = clf.fit(scale(X), Y)
predictions = model.predict(scale(X))
accuracy_score(Y, predictions)

In [ ]:
clf = sklearn.svm.SVC(kernel='poly')
model = clf.fit(scaling_to_zero_one(X), Y)
predictions = model.predict(scaling_to_zero_one(X))
accuracy_score(Y, predictions)

In [ ]:
clf = sklearn.svm.SVC(kernel='sigmoid')
model = clf.fit(X, Y)
predictions = model.predict(X)
accuracy_score(Y, predictions)

In [ ]:
clf = sklearn.svm.SVC(kernel='sigmoid')
model = clf.fit(scale(X), Y)
predictions = model.predict(scale(X))
accuracy_score(Y, predictions)

In [ ]:
clf = sklearn.svm.SVC(kernel='sigmoid')
model = clf.fit(scaling_to_zero_one(X), Y)
predictions = model.predict(scaling_to_zero_one(X))
accuracy_score(Y, predictions)

### **Задание 2**

Подберите параметры для SVC, дающие наибольшую точность (accuracy score)

### Анализ результатов

Прогнозирование:

In [ ]:
df=pd.concat([X, pd.DataFrame(predictions, columns=['predictions'])], axis=1)
sns.set_style('darkgrid')
sns.lmplot('petal length', 'petal width', df, hue='predictions', scatter=True, fit_reg=False)

**Матрица ошибок (Confusion matrix)**

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

sns.set_style('white')

def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


cnf_matrix = confusion_matrix(Y, predictions)
np.set_printoptions(precision=2)

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=classes,\
                      title='Confusion matrix, without normalization')

plt.show()